## Grad Boost IMPLEMENTATION

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Define the Gradient Boosting model
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
def evaluate(model, X, y):
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)
    return accuracy, precision, recall, f1

# Evaluate on training data
train_accuracy, train_precision, train_recall, train_f1 = evaluate(model, X_train, y_train)
print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Train - Accuracy: 0.9528, Precision: 0.9498, Recall: 0.9659, F1: 0.9578
Test - Accuracy: 0.9507, Precision: 0.9491, Recall: 0.9649, F1: 0.9569


## SVM( LINEAR  ) IMPLEMENTATION

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Define the SVM model
model = SVC(kernel='linear', C=1, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
def evaluate(model, X, y):
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)
    return accuracy, precision, recall, f1

# Evaluate on training data
train_accuracy, train_precision, train_recall, train_f1 = evaluate(model, X_train, y_train)
print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Train - Accuracy: 0.9284, Precision: 0.9263, Recall: 0.9461, F1: 0.9361
Test - Accuracy: 0.9285, Precision: 0.9262, Recall: 0.9498, F1: 0.9378


## CNN IMPLEMENTATION

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * (X_train.shape[2] // 4), 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and the optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train_model()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.6136
Train - Accuracy: 0.7353, Precision: 0.6795, Recall: 0.9890, F1: 0.8055
Epoch: 020, Loss: 0.4629
Train - Accuracy: 0.8414, Precision: 0.8318, Recall: 0.8947, F1: 0.8621
Epoch: 030, Loss: 0.3162
Train - Accuracy: 0.8779, Precision: 0.8862, Recall: 0.8945, F1: 0.8904
Epoch: 040, Loss: 0.2525
Train - Accuracy: 0.8956, Precision: 0.9049, Recall: 0.9070, F1: 0.9060
Epoch: 050, Loss: 0.2166
Train - Accuracy: 0.9101, Precision: 0.9134, Recall: 0.9255, F1: 0.9194
Epoch: 060, Loss: 0.1950
Train - Accuracy: 0.9227, Precision: 0.9216, Recall: 0.9404, F1: 0.9309
Epoch: 070, Loss: 0.1819
Train - Accuracy: 0.9289, Precision: 0.9264, Recall: 0.9470, F1: 0.9365
Epoch: 080, Loss: 0.1720
Train - Accuracy: 0.9326, Precision: 0.9318, Recall: 0.9478, F1: 0.9397
Epoch: 090, Loss: 0.1640
Train - Accuracy: 0.9348, Precision: 0.9333, Recall: 0.9502, F1: 0.9417
Epoch: 100, Loss: 0.1572
Train - Accuracy: 0.9366, Precision: 0.9351, Recall: 0.9517, F1: 0.9433
Epoch: 110, Loss: 0.1509
Train

## CNN Leaky Relu

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Add a channel dimension
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the CNN model with LeakyReLU
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)
        self.fc1 = nn.Linear(32 * (X_train.shape[2] // 4), 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(self.leaky_relu(self.conv1(x)))
        x = self.pool(self.leaky_relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and the optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train_model()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.6390
Train - Accuracy: 0.6203, Precision: 0.5936, Recall: 0.9986, F1: 0.7446
Epoch: 020, Loss: 0.5150
Train - Accuracy: 0.8217, Precision: 0.7955, Recall: 0.9131, F1: 0.8502
Epoch: 030, Loss: 0.3543
Train - Accuracy: 0.8786, Precision: 0.8741, Recall: 0.9123, F1: 0.8928
Epoch: 040, Loss: 0.2661
Train - Accuracy: 0.8950, Precision: 0.8974, Recall: 0.9151, F1: 0.9062
Epoch: 050, Loss: 0.2270
Train - Accuracy: 0.9101, Precision: 0.9138, Recall: 0.9251, F1: 0.9194
Epoch: 060, Loss: 0.2013
Train - Accuracy: 0.9198, Precision: 0.9159, Recall: 0.9419, F1: 0.9287
Epoch: 070, Loss: 0.1849
Train - Accuracy: 0.9267, Precision: 0.9225, Recall: 0.9474, F1: 0.9348
Epoch: 080, Loss: 0.1741
Train - Accuracy: 0.9288, Precision: 0.9265, Recall: 0.9466, F1: 0.9364
Epoch: 090, Loss: 0.1650
Train - Accuracy: 0.9310, Precision: 0.9289, Recall: 0.9482, F1: 0.9384
Epoch: 100, Loss: 0.1567
Train - Accuracy: 0.9336, Precision: 0.9319, Recall: 0.9496, F1: 0.9407
Epoch: 110, Loss: 0.1494
Train

## KNN Classifier

In [19]:
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train.numpy(), y_train.numpy())

# Make predictions on the train set
y_train_pred = knn.predict(X_train.numpy())

# Make predictions on the test set
y_test_pred = knn.predict(X_test.numpy())

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train.numpy(), y_train_pred)
train_precision = precision_score(y_train.numpy(), y_train_pred)
train_recall = recall_score(y_train.numpy(), y_train_pred)
train_f1 = f1_score(y_train.numpy(), y_train_pred)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test.numpy(), y_test_pred)
test_precision = precision_score(y_test.numpy(), y_test_pred)
test_recall = recall_score(y_test.numpy(), y_test_pred)
test_f1 = f1_score(y_test.numpy(), y_test_pred)

print(f'KNN Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
print(f'KNN Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


KNN Train - Accuracy: 0.9665, Precision: 0.9658, Recall: 0.9741, F1: 0.9699
KNN Test - Accuracy: 0.9408, Precision: 0.9411, Recall: 0.9554, F1: 0.9482


## Logistic Regression

In [17]:
import torch.nn as nn
import torch.optim as optim

# Define the Logistic Regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 2)

    def forward(self, x):
        return self.linear(x)

# Instantiate the model, define the loss function and the optimizer
model = LogisticRegression(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train_model()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.8541
Train - Accuracy: 0.3405, Precision: 0.4075, Recall: 0.4186, F1: 0.4130
Epoch: 020, Loss: 0.8024
Train - Accuracy: 0.3823, Precision: 0.4450, Recall: 0.4629, F1: 0.4538
Epoch: 030, Loss: 0.7564
Train - Accuracy: 0.4602, Precision: 0.5121, Recall: 0.5510, F1: 0.5309
Epoch: 040, Loss: 0.7149
Train - Accuracy: 0.5286, Precision: 0.5669, Recall: 0.6336, F1: 0.5984
Epoch: 050, Loss: 0.6777
Train - Accuracy: 0.5932, Precision: 0.6171, Recall: 0.7007, F1: 0.6563
Epoch: 060, Loss: 0.6445
Train - Accuracy: 0.6451, Precision: 0.6581, Recall: 0.7487, F1: 0.7004
Epoch: 070, Loss: 0.6149
Train - Accuracy: 0.6825, Precision: 0.6876, Recall: 0.7829, F1: 0.7322
Epoch: 080, Loss: 0.5885
Train - Accuracy: 0.7159, Precision: 0.7125, Recall: 0.8170, F1: 0.7612
Epoch: 090, Loss: 0.5647
Train - Accuracy: 0.7417, Precision: 0.7367, Recall: 0.8311, F1: 0.7811
Epoch: 100, Loss: 0.5433
Train - Accuracy: 0.7655, Precision: 0.7583, Recall: 0.8468, F1: 0.8001
Epoch: 110, Loss: 0.5240
Train

## Naive Bayes

In [18]:
import torch
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Initialize and train the Naive Bayes classifier
nb = GaussianNB()
nb.fit(X_train.numpy(), y_train.numpy())

# Make predictions on the train set
y_train_pred = nb.predict(X_train.numpy())

# Make predictions on the test set
y_test_pred = nb.predict(X_test.numpy())

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train.numpy(), y_train_pred)
train_precision = precision_score(y_train.numpy(), y_train_pred)
train_recall = recall_score(y_train.numpy(), y_train_pred)
train_f1 = f1_score(y_train.numpy(), y_train_pred)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test.numpy(), y_test_pred)
test_precision = precision_score(y_test.numpy(), y_test_pred)
test_recall = recall_score(y_test.numpy(), y_test_pred)
test_f1 = f1_score(y_test.numpy(), y_test_pred)

print(f'Naive Bayes Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
print(f'Naive Bayes Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Naive Bayes Train - Accuracy: 0.6030, Precision: 0.9957, Recall: 0.2850, F1: 0.4431
Naive Bayes Test - Accuracy: 0.5798, Precision: 0.9970, Recall: 0.2606, F1: 0.4131


## RESNET

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the ResNet block
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        out = F.relu(out)
        return out

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, num_features, num_classes):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(num_features, 64)
        self.block1 = ResidualBlock(64)
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.output_layer = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)
        return F.log_softmax(x, dim=1)

# Model parameters
num_features = X_train.shape[1]
num_classes = 2  # Phishing or not

# Instantiate the model
model = ResNet(num_features, num_classes)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training the model
def train():
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate(data, labels):
    model.eval()
    with torch.no_grad():
        output = model(data)
        pred = output.argmax(dim=1)
        accuracy = accuracy_score(labels.cpu(), pred.cpu())
        precision = precision_score(labels.cpu(), pred.cpu())
        recall = recall_score(labels.cpu(), pred.cpu())
        f1 = f1_score(labels.cpu(), pred.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 010, Loss: 0.1572
Train - Accuracy: 0.9268, Precision: 0.9122, Recall: 0.9604, F1: 0.9357
Epoch: 020, Loss: 0.1197
Train - Accuracy: 0.9439, Precision: 0.9311, Recall: 0.9706, F1: 0.9505
Epoch: 030, Loss: 0.0793
Train - Accuracy: 0.9638, Precision: 0.9528, Recall: 0.9835, F1: 0.9679
Epoch: 040, Loss: 0.0474
Train - Accuracy: 0.9729, Precision: 0.9594, Recall: 0.9931, F1: 0.9759
Epoch: 050, Loss: 0.0304
Train - Accuracy: 0.9773, Precision: 0.9633, Recall: 0.9969, F1: 0.9798
Epoch: 060, Loss: 0.0228
Train - Accuracy: 0.9885, Precision: 0.9831, Recall: 0.9963, F1: 0.9897
Epoch: 070, Loss: 0.0208
Train - Accuracy: 0.9895, Precision: 0.9906, Recall: 0.9904, F1: 0.9905
Epoch: 080, Loss: 0.0195
Train - Accuracy: 0.9902, Precision: 0.9869, Recall: 0.9955, F1: 0.9912
Epoch: 090, Loss: 0.0189
Train - Accuracy: 0.9905, Precision: 0.9884, Recall: 0.9945, F1: 0.9915
Epoch: 100, Loss: 0.0181
Train - Accuracy: 0.9902, Precision: 0.9892, Recall: 0.9931, F1: 0.9911
Epoch: 110, Loss: 0.0230
Train

### 6 TYPES OF RESNETS

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the ResNet block for different variants
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        out = F.relu(out)
        return out

# Define the ResNet model for different variants
class ResNet(nn.Module):
    def __init__(self, num_features, num_classes, block, layers):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(num_features, 64)
        self.layer1 = self.make_layer(block, 64, layers[0])
        self.layer2 = self.make_layer(block, 64, layers[1])
        self.layer3 = self.make_layer(block, 64, layers[2])
        self.output_layer = nn.Linear(64, num_classes)

    def make_layer(self, block, in_features, blocks):
        layers = []
        for _ in range(blocks):
            layers.append(block(in_features))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output_layer(x)
        return F.log_softmax(x, dim=1)

# Model parameters
num_features = X_train.shape[1]
num_classes = 2  # Phishing or not
layers = [3, 3, 3]  # Number of layers in each block

# Instantiate and train different ResNet variants
variants = ["ResNet", "ResNetV2", "PreActResNet", "WideResNet", "ResNeXt", "DenseNet"]
for variant in variants:
    print(f"Training {variant}...")
    model = ResNet(num_features, num_classes, ResidualBlock, layers)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(1, 201):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                output = model(X_train)
                pred = output.argmax(dim=1)
                train_accuracy = accuracy_score(y_train.cpu(), pred.cpu())
                train_precision = precision_score(y_train.cpu(), pred.cpu())
                train_recall = recall_score(y_train.cpu(), pred.cpu())
                train_f1 = f1_score(y_train.cpu(), pred.cpu())
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
            print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

    # Evaluate on test data
    model.eval()
    with torch.no_grad():
        output = model(X_test)
        pred = output.argmax(dim=1)
        test_accuracy = accuracy_score(y_test.cpu(), pred.cpu())
        test_precision = precision_score(y_test.cpu(), pred.cpu())
        test_recall = recall_score(y_test.cpu(), pred.cpu())
        test_f1 = f1_score(y_test.cpu(), pred.cpu())
    print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')
    print()


Training ResNet...
Epoch: 010, Loss: 0.1789
Train - Accuracy: 0.9244, Precision: 0.9392, Recall: 0.9233, F1: 0.9312
Epoch: 020, Loss: 0.1315
Train - Accuracy: 0.9439, Precision: 0.9550, Recall: 0.9433, F1: 0.9491
Epoch: 030, Loss: 0.1003
Train - Accuracy: 0.9557, Precision: 0.9613, Recall: 0.9586, F1: 0.9600
Epoch: 040, Loss: 0.0678
Train - Accuracy: 0.9703, Precision: 0.9659, Recall: 0.9810, F1: 0.9734
Epoch: 050, Loss: 0.0524
Train - Accuracy: 0.9802, Precision: 0.9743, Recall: 0.9904, F1: 0.9823
Epoch: 060, Loss: 0.0340
Train - Accuracy: 0.9869, Precision: 0.9823, Recall: 0.9943, F1: 0.9882
Epoch: 070, Loss: 0.0243
Train - Accuracy: 0.9884, Precision: 0.9896, Recall: 0.9894, F1: 0.9895
Epoch: 080, Loss: 0.0203
Train - Accuracy: 0.9874, Precision: 0.9920, Recall: 0.9853, F1: 0.9886
Epoch: 090, Loss: 0.0188
Train - Accuracy: 0.9891, Precision: 0.9890, Recall: 0.9914, F1: 0.9902
Epoch: 100, Loss: 0.0312
Train - Accuracy: 0.9887, Precision: 0.9872, Recall: 0.9925, F1: 0.9898
Epoch: 110,

In [34]:
class ResidualBlockV2(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlockV2, self).__init__()
        self.layer = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        return out


In [35]:
class PreActResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(PreActResidualBlock, self).__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(in_features, in_features)
        self.bn2 = nn.BatchNorm1d(in_features)
        self.fc2 = nn.Linear(in_features, in_features)

    def forward(self, x):
        out = self.relu(self.bn1(x))
        out = self.fc1(out)
        out = self.relu(self.bn2(out))
        out = self.fc2(out)
        out += x
        return out


In [36]:
class PreActResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(PreActResidualBlock, self).__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(in_features, in_features)
        self.bn2 = nn.BatchNorm1d(in_features)
        self.fc2 = nn.Linear(in_features, in_features)

    def forward(self, x):
        out = self.relu(self.bn1(x))
        out = self.fc1(out)
        out = self.relu(self.bn2(out))
        out = self.fc2(out)
        out += x
        return out


In [37]:
class WideResidualBlock(nn.Module):
    def __init__(self, in_features, widen_factor=2):
        super(WideResidualBlock, self).__init__()
        widened_features = in_features * widen_factor
        self.layer = nn.Sequential(
            nn.Linear(in_features, widened_features),
            nn.BatchNorm1d(widened_features),
            nn.ReLU(inplace=True),
            nn.Linear(widened_features, in_features),
            nn.BatchNorm1d(in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        out = F.relu(out)
        return out


In [39]:
class DenseBlock(nn.Module):
    def __init__(self, in_features, growth_rate=32):
        super(DenseBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, growth_rate)
        )

    def forward(self, x):
        out = self.layer(x)
        out = torch.cat([x, out], dim=1)
        return out


In [38]:
class ResNeXtBlock(nn.Module):
    def __init__(self, in_features, cardinality=4, bottleneck_width=64):
        super(ResNeXtBlock, self).__init__()
        D = cardinality * bottleneck_width
        self.layer = nn.Sequential(
            nn.Linear(in_features, D),
            nn.BatchNorm1d(D),
            nn.ReLU(inplace=True),
            nn.Linear(D, in_features),
            nn.BatchNorm1d(in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        out = F.relu(out)
        return out


In [40]:
import matplotlib.pyplot as plt

# Define the training and evaluation functions
def train(model, optimizer, criterion, X_train, y_train):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, X_data, y_data):
    model.eval()
    with torch.no_grad():
        output = model(X_data)
        pred = output.argmax(dim=1)
        accuracy = accuracy_score(y_data.cpu(), pred.cpu())
        precision = precision_score(y_data.cpu(), pred.cpu())
        recall = recall_score(y_data.cpu(), pred.cpu())
        f1 = f1_score(y_data.cpu(), pred.cpu())
    return accuracy, precision, recall, f1

# Function to train and evaluate a model
def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=200):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    train_losses = []
    train_accuracies = []
    train_precisions = []
    train_recalls = []
    train_f1s = []

    for epoch in range(1, epochs + 1):
        loss = train(model, optimizer, criterion, X_train, y_train)
        train_losses.append(loss)

        if epoch % 10 == 0:
            train_accuracy, train_precision, train_recall, train_f1 = evaluate(model, X_train, y_train)
            train_accuracies.append(train_accuracy)
            train_precisions.append(train_precision)
            train_recalls.append(train_recall)
            train_f1s.append(train_f1)
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
            print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

    test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, X_test, y_test)
    print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')

    return train_losses, train_accuracies, train_precisions, train_recalls, train_f1s, test_accuracy, test_precision, test_recall, test_f1

# Store the results for each variant
results = {}

# List of variants to train
variants = ["ResNet", "ResNetV2", "PreActResNet", "WideResNet", "ResNeXt", "DenseNet"]

for variant in variants:
    print(f"Training {variant}...")
    if variant == "ResNet":
        model = ResNet(num_features, num_classes, ResidualBlock, layers)
    elif variant == "ResNetV2":
        model = ResNet(num_features, num_classes, ResidualBlockV2, layers)
    elif variant == "PreActResNet":
        model = PreActResNet(num_features, num_classes, PreActResidualBlock, layers)
    elif variant == "WideResNet":
        model = WideResNet(num_features, num_classes, WideResidualBlock, layers)
    elif variant == "ResNeXt":
        model = ResNeXt(num_features, num_classes, ResNeXtBlock, layers)
    elif variant == "DenseNet":
        model = DenseNet(num_features, num_classes, DenseBlock, layers)

    losses, accuracies, precisions, recalls, f1s, test_acc, test_prec, test_rec, test_f1 = train_and_evaluate(model, X_train, y_train, X_test, y_test)
    results[variant] = {
        'losses': losses,
        'accuracies': accuracies,
        'precisions': precisions,
        'recalls': recalls,
        'f1s': f1s,
        'test_acc': test_acc,
        'test_prec': test_prec,
        'test_rec': test_rec,
        'test_f1': test_f1
    }

# Plot the results
plt.figure(figsize=(15, 10))

# Plot training loss
plt.subplot(2, 2, 1)
for variant in variants:
    plt.plot(results[variant]['losses'], label=variant)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot training accuracy
plt.subplot(2, 2, 2)
for variant in variants:
    plt.plot(range(10, 201, 10), results[variant]['accuracies'], label=variant)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()

# Plot training precision
plt.subplot(2, 2, 3)
for variant in variants:
    plt.plot(range(10, 201, 10), results[variant]['precisions'], label=variant)
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.title('Training Precision')
plt.legend()

# Plot training F1 score
plt.subplot(2, 2, 4)
for variant in variants:
    plt.plot(range(10, 201, 10), results[variant]['f1s'], label=variant)
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Training F1 Score')
plt.legend()

plt.tight_layout()
plt.show()


Training ResNet...
Epoch: 010, Loss: 0.1730
Train - Accuracy: 0.8792, Precision: 0.9597, Recall: 0.8164, F1: 0.8823
Epoch: 020, Loss: 0.1194
Train - Accuracy: 0.9311, Precision: 0.9631, Recall: 0.9106, F1: 0.9361
Epoch: 030, Loss: 0.0804
Train - Accuracy: 0.9631, Precision: 0.9683, Recall: 0.9651, F1: 0.9667
Epoch: 040, Loss: 0.0497
Train - Accuracy: 0.9770, Precision: 0.9780, Recall: 0.9806, F1: 0.9793
Epoch: 050, Loss: 0.0349
Train - Accuracy: 0.9833, Precision: 0.9762, Recall: 0.9941, F1: 0.9850
Epoch: 060, Loss: 0.0252
Train - Accuracy: 0.9885, Precision: 0.9847, Recall: 0.9947, F1: 0.9896
Epoch: 070, Loss: 0.0209
Train - Accuracy: 0.9898, Precision: 0.9882, Recall: 0.9935, F1: 0.9908
Epoch: 080, Loss: 0.0189
Train - Accuracy: 0.9905, Precision: 0.9894, Recall: 0.9935, F1: 0.9914
Epoch: 090, Loss: 0.0189
Train - Accuracy: 0.9893, Precision: 0.9912, Recall: 0.9894, F1: 0.9903
Epoch: 100, Loss: 0.0181
Train - Accuracy: 0.9904, Precision: 0.9876, Recall: 0.9951, F1: 0.9914
Epoch: 110,

NameError: name 'PreActResNet' is not defined

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Reshape the data to add a channel dimension and an extra spatial dimension
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1).unsqueeze(2)  # Shape: [batch_size, channels, height, width]
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1).unsqueeze(2)  # Shape: [batch_size, channels, height, width]
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the MobileNet model
class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.mobilenet.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
        self.mobilenet.classifier[1] = nn.Linear(self.mobilenet.last_channel, 2)

    def forward(self, x):
        x = self.mobilenet(x)
        return x

# Instantiate the model, define the loss function and the optimizer
model = MobileNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 21):  # Reduced number of epochs for time consideration
    loss = train_model()
    if epoch % 2 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 002, Loss: 0.5287
Train - Accuracy: 0.4457, Precision: 0.0000, Recall: 0.0000, F1: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 004, Loss: 0.3205
Train - Accuracy: 0.4457, Precision: 0.0000, Recall: 0.0000, F1: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 006, Loss: 0.2193
Train - Accuracy: 0.4457, Precision: 0.0000, Recall: 0.0000, F1: 0.0000


KeyboardInterrupt: 

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the simple feedforward neural network
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = self.leaky_relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, define the loss function and the optimizer
input_dim = X_train.shape[1]
model = FeedForwardNN(input_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu())
        recall = recall_score(labels.cpu(), predicted.cpu())
        f1 = f1_score(labels.cpu(), predicted.cpu())
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 21):  # Reduced number of epochs for time consideration
    loss = train_model()
    if epoch % 2 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Epoch: 002, Loss: 0.6919
Train - Accuracy: 0.6124, Precision: 0.6126, Recall: 0.8182, F1: 0.7006
Epoch: 004, Loss: 0.6612
Train - Accuracy: 0.7101, Precision: 0.6797, Recall: 0.9019, F1: 0.7752
Epoch: 006, Loss: 0.6322
Train - Accuracy: 0.7612, Precision: 0.7199, Recall: 0.9317, F1: 0.8122
Epoch: 008, Loss: 0.6029
Train - Accuracy: 0.7886, Precision: 0.7459, Recall: 0.9380, F1: 0.8310
Epoch: 010, Loss: 0.5722
Train - Accuracy: 0.8117, Precision: 0.7711, Recall: 0.9392, F1: 0.8469
Epoch: 012, Loss: 0.5397
Train - Accuracy: 0.8353, Precision: 0.8002, Recall: 0.9366, F1: 0.8631
Epoch: 014, Loss: 0.5052
Train - Accuracy: 0.8541, Precision: 0.8256, Recall: 0.9341, F1: 0.8765
Epoch: 016, Loss: 0.4696
Train - Accuracy: 0.8710, Precision: 0.8500, Recall: 0.9317, F1: 0.8890
Epoch: 018, Loss: 0.4335
Train - Accuracy: 0.8814, Precision: 0.8659, Recall: 0.9300, F1: 0.8968
Epoch: 020, Loss: 0.3980
Train - Accuracy: 0.8892, Precision: 0.8790, Recall: 0.9278, F1: 0.9027
Test - Accuracy: 0.8851, Preci

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Check class distribution
print(dataset['Result'].value_counts())

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the feedforward neural network with ReLU and Leaky ReLU layers
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.leaky_relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.fc6(x)
        return x

# Instantiate the model, define the loss function and the optimizer
input_dim = X_train.shape[1]
model = FeedForwardNN(input_dim)

# Calculate class weights to handle imbalance
class_counts = dataset['Result'].value_counts().to_dict()
class_weights = {cls: 1.0/count for cls, count in class_counts.items()}
weights = torch.tensor([class_weights[cls] for cls in range(2)], dtype=torch.float32)
criterion = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model():
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate_model(data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(labels.cpu(), predicted.cpu())
        precision = precision_score(labels.cpu(), predicted.cpu(), zero_division=1)
        recall = recall_score(labels.cpu(), predicted.cpu(), zero_division=1)
        f1 = f1_score(labels.cpu(), predicted.cpu(), zero_division=1)
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 51):  # Increased number of epochs
    loss = train_model()
    if epoch % 2 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')


Result
1    6157
0    4898
Name: count, dtype: int64
Epoch: 002, Loss: 0.6956
Train - Accuracy: 0.5543, Precision: 0.5543, Recall: 1.0000, F1: 0.7132
Epoch: 004, Loss: 0.6915
Train - Accuracy: 0.5543, Precision: 0.5543, Recall: 1.0000, F1: 0.7132
Epoch: 006, Loss: 0.6861
Train - Accuracy: 0.5543, Precision: 0.5543, Recall: 1.0000, F1: 0.7132
Epoch: 008, Loss: 0.6784
Train - Accuracy: 0.5559, Precision: 0.5552, Recall: 1.0000, F1: 0.7140
Epoch: 010, Loss: 0.6678
Train - Accuracy: 0.6098, Precision: 0.5870, Recall: 0.9986, F1: 0.7394
Epoch: 012, Loss: 0.6529
Train - Accuracy: 0.8282, Precision: 0.7710, Recall: 0.9816, F1: 0.8637
Epoch: 014, Loss: 0.6312
Train - Accuracy: 0.8883, Precision: 0.8520, Recall: 0.9663, F1: 0.9056
Epoch: 016, Loss: 0.5996
Train - Accuracy: 0.9100, Precision: 0.8937, Recall: 0.9506, F1: 0.9213
Epoch: 018, Loss: 0.5544
Train - Accuracy: 0.9197, Precision: 0.9146, Recall: 0.9433, F1: 0.9287
Epoch: 020, Loss: 0.4933
Train - Accuracy: 0.9228, Precision: 0.9285, Reca

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Load the dataset
dataset = pd.read_csv('dataset.csv')

# Drop the index column
dataset = dataset.drop(columns=['index'])

# Convert target labels -1 to 0
dataset['Result'] = dataset['Result'].replace(-1, 0)

# Check class distribution
print(dataset['Result'].value_counts())

# Split the dataset into features and target
X = dataset.drop(columns=['Result'])
y = dataset['Result']

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define the ResNet block
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features),
            nn.ReLU(inplace=True),
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features)
        )

    def forward(self, x):
        residual = x
        out = self.layer(x)
        out += residual
        out = F.relu(out)
        return out

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, num_features, num_classes):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(num_features, 64)
        self.block1 = ResidualBlock(64)
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.output_layer = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)
        return x

# Model parameters
num_features = X_train.shape[1]
num_classes = 2  # Phishing or not

# Instantiate the model
model = ResNet(num_features, num_classes)

# Calculate class weights to handle imbalance
class_counts = dataset['Result'].value_counts().to_dict()
class_weights = {cls: 1.0/count for cls, count in class_counts.items()}
weights = torch.tensor([class_weights[cls] for cls in range(2)], dtype=torch.float32)
criterion = nn.CrossEntropyLoss(weight=weights)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training the model
def train():
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate(data, labels):
    model.eval()
    with torch.no_grad():
        output = model(data)
        pred = output.argmax(dim=1)
        accuracy = accuracy_score(labels.cpu(), pred.cpu())
        precision = precision_score(labels.cpu(), pred.cpu(), zero_division=1)
        recall = recall_score(labels.cpu(), pred.cpu(), zero_division=1)
        f1 = f1_score(labels.cpu(), pred.cpu(), zero_division=1)
    return accuracy, precision, recall, f1

# Training loop
for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        train_accuracy, train_precision, train_recall, train_f1 = evaluate(X_train, y_train)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Evaluate on test data
test_accuracy, test_precision, test_recall, test_f1 = evaluate(X_test, y_test)
print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')

Result
1    6157
0    4898
Name: count, dtype: int64
Epoch: 010, Loss: 0.1468
Train - Accuracy: 0.9387, Precision: 0.9381, Recall: 0.9523, F1: 0.9451
Epoch: 020, Loss: 0.0938
Train - Accuracy: 0.9544, Precision: 0.9440, Recall: 0.9757, F1: 0.9596
Epoch: 030, Loss: 0.0530
Train - Accuracy: 0.9691, Precision: 0.9521, Recall: 0.9943, F1: 0.9728
Epoch: 040, Loss: 0.0378
Train - Accuracy: 0.9783, Precision: 0.9639, Recall: 0.9982, F1: 0.9808
Epoch: 050, Loss: 0.0250
Train - Accuracy: 0.9874, Precision: 0.9796, Recall: 0.9982, F1: 0.9888
Epoch: 060, Loss: 0.0214
Train - Accuracy: 0.9900, Precision: 0.9869, Recall: 0.9953, F1: 0.9911
Epoch: 070, Loss: 0.0194
Train - Accuracy: 0.9899, Precision: 0.9867, Recall: 0.9953, F1: 0.9910
Epoch: 080, Loss: 0.0188
Train - Accuracy: 0.9900, Precision: 0.9922, Recall: 0.9898, F1: 0.9910
Epoch: 090, Loss: 0.0183
Train - Accuracy: 0.9904, Precision: 0.9890, Recall: 0.9937, F1: 0.9914
Epoch: 100, Loss: 0.0198
Train - Accuracy: 0.9906, Precision: 0.9896, Reca

NameError: name 's' is not defined